In [1]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.constants import c, hbar
import sys
sys.path.insert(0, "../")
# Custom modules
from plotting import makefig
import spectral_intensity as si
from v_x_interp import position_interp, velocity_interp

In [7]:
# Load multi-particle data from wargSim
L_plasma = 0.4
path     = "/media/keenan/Data_Storage/WARGSim/" + "noGain/"
nDumps   = 699
nParts   = 100
z        = np.linspace(0, L_plasma, nDumps)
x        = np.zeros((nParts, nDumps))
y        = np.zeros((nParts, nDumps))
gb       = np.zeros((nParts, nDumps))
t        = z / c;
t[0]     = 0;
for i in range(nDumps):
    if i%100 == 0 and i >=100:
        print(i)
    ptcls    = np.load(path + "_ptcls_" + str(i) + ".npy")
    x[:, i]  = ptcls[0:nParts,0]
    y[:, i]  = ptcls[0:nParts, 2]
    gb[:, i] = ptcls[0, 5] 

100
200
300
400
500
600


In [8]:
# Get velocities
def get_v(x, y, z, tau):
    vx  = np.diff(x) / np.diff(tau)
    vx  = np.append(vx, vx[-1])
    vy  = np.diff(y) / np.diff(tau)
    vy  = np.append(vy, vy[-1])
    vz  = np.diff(z) / np.diff(tau)
    vz  = np.append(vz, vz[-1])
    return vx, vy, vz
# get tau
def get_tau(dtau):
    tau = np.zeros(len(dtau))
    for i in range(1,len(dtau)):
        tau[i] = tau[i-1] + dtau[i-1]
    return tau

In [ ]:
# Loop through angles, energies, and particles
dt       = t[1] - t[0]
N_interp = 1000
npad = -3
theta = np.linspace(-np.pi/2, np.pi/2, 500)
E     = np.linspace(1e-6, 10, 500) * 1e3 # E, eV
w     = E / hbar
d2I   = np.zeros((len(w), len(theta)), dtype = 'complex128')
for i in range(len(w)):
    if (i+1)%100 == 0:
        print(i+1)
    w_use = w[i]
    for j in range(len(theta)):
        for k in range(len(x)):
            x_use  = x[k, :]
            y_use  = y[k, :]
            gb_use = gb[k, :]
            dtau   = dt / gb_use
            tau    = get_tau(dtau)
            vx, vy, vz = get_v(x_use, y_use, z, tau)
            # Interpolate
            tau_int = np.linspace(tau[0], tau[-1], N_interp)
            x0n     = (c * t, x_use, y_use, z)
            v0n     = (vx, vy, vz)
            ct1, ct2, ct_int = position_interp(x0n[0], tau, tau_int)
            x1, x2, x_int    = position_interp(x0n[1], tau, tau_int)
            y1, y2, y_int    = position_interp(x0n[2], tau, tau_int)
            z1, z2, z_int    = position_interp(x0n[3], tau, tau_int)
            x0n              = (x0n[0][0:npad], x0n[1][0:npad], x0n[2][0:npad], x0n[3][0:npad])
            x1n              = (ct1[0:npad], x1[0:npad], y1[0:npad], z1[0:npad])
            x2n              = (ct2[0:npad], x2[0:npad], y2[0:npad], z2[0:npad])

            # Interpolate velocity
            vx1, vx_int = velocity_interp(v0n[0], tau, tau_int)
            vy1, vy_int = velocity_interp(v0n[1], tau, tau_int)
            vz1, vz_int = velocity_interp(v0n[2], tau, tau_int)

            v0n         = (v0n[0][0:npad], v0n[1][0:npad], v0n[2][0:npad])
            v1n         = (vx1[0:npad], vy1[0:npad], vz1[0:npad]) 

            tau  = tau[0:npad]  
            dtau = dtau[0:npad]
            d2I[i, j] += si.get_single_intensity(theta[j], w_use, x0n, x1n, x2n, v0n, v1n, dtau)

100
200


In [ ]:
fig, ax = makefig(xlab = r'E [keV]', ylab = r'$\theta [rad]$')
d2I_plot = np.real(d2I) / np.amax(np.real(d2I))
ext  = [min(E), max(E), min(theta), max(theta)]
img  = ax.imshow(np.transpose(d2I_plot), extent = ext, cmap = 'jet', aspect = 'auto')
cb   = plt.colorbar(mappable = img)
plt.show()

In [2]:
L_plasma = 0.4, # plasma length from simulation (m)
z = np.squeeze(np.linspace(0, L_plasma, len(x)))
t = z / c;
# Get proper time
dtau = (t[1] - t[0]) / gb
tau  = np.zeros(len(t))
for i in range(1, len(t)):
    tau[i] = tau[i-1] + dtau[i-1]
x0n = (c * t, x, y, z)
# Get velocities
vx  = np.diff(x) / np.diff(tau)
vx  = np.append(vx, vx[-1])
vy  = np.diff(y) / np.diff(tau)
vy  = np.append(vy, vy[-1])
vz  = np.diff(z) / np.diff(tau)
vz  = np.append(vz, vz[-1])

v0n = (vx, vy, vz)

In [ ]:
N_interp = 1000 # Number of interpolation points
tau_int = np.linspace(tau[0], tau[-1], N_interp)

In [3]:
# Get interpolation coefficients as vectors
N_interp = 1000 # Number of interpolation points
tau_int = np.linspace(tau[0], tau[-1], N_interp)

# Interp osition
ct1, ct2, ct_int = position_interp(x0n[0], tau, tau_int)
x1, x2, x_int    = position_interp(x0n[1], tau, tau_int)
y1, y2, y_int    = position_interp(x0n[2], tau, tau_int)
z1, z2, z_int    = position_interp(x0n[3], tau, tau_int)

# Testing padding the results
npad = -3
x0n              = (x0n[0][0:npad], x0n[1][0:npad], x0n[2][0:npad], \
                    x0n[3][0:npad])
x1n              = (ct1[0:npad], x1[0:npad], y1[0:npad], z1[0:npad])
x2n              = (ct2[0:npad], x2[0:npad], y2[0:npad], z2[0:npad])

# Interpolate velocity
vx1, vx_int = velocity_interp(v0n[0], tau, tau_int)
vy1, vy_int = velocity_interp(v0n[1], tau, tau_int)
vz1, vz_int = velocity_interp(v0n[2], tau, tau_int)

v0n         = (v0n[0][0:npad], v0n[1][0:npad], v0n[2][0:npad])
v1n         = (vx1[0:npad], vy1[0:npad], vz1[0:npad]) 

tau  = tau[0:npad]  
dtau = dtau[0:npad]

100
200
300
400
500
